<a href="https://colab.research.google.com/github/AI4ALL-ESG-Investing/esg-financial-assistant/blob/main/data-cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning Notebook

# Import python libraries

In [ ]:
import os
import pandas as pd
import yfinance as yf

## Load Data into Colab

In [10]:
# Clone the GitHub repository
!git clone https://github.com/AI4ALL-ESG-Investing/esg-financial-assistant

# Change working directory to where the dataset is located
%cd esg-financial-assistant/data

os.listdir()

Cloning into 'esg-financial-assistant'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 20 (delta 3), reused 14 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 56.54 KiB | 4.04 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/esg-financial-assistant/data/esg-financial-assistant/data


['public-company-esg-ratings-dataset.csv', 'investment_survey.csv']

In [31]:
# clean ESG dataset
df = pd.read_csv("public-company-esg-ratings-dataset.csv")
df.head()

,ticker,name,currency,exchange,industry,logo,weburl,environment_grade,environment_level,social_grade,...,governance_grade,governance_level,environment_score,social_score,governance_score,total_score,last_processing_date,total_grade,total_level,cik
0,dis,Walt Disney Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,https://static.finnhub.io/logo/ef50b4a2b263c84...,https://thewaltdisneycompany.com/,A,High,BB,...,BB,Medium,510,316,321,1147,19-04-2022,BBB,High,1744489
1,gm,General Motors Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Automobiles,https://static.finnhub.io/logo/9253db78-80c9-1...,https://www.gm.com/,A,High,BB,...,B,Medium,510,303,255,1068,17-04-2022,BBB,High,1467858
2,gww,WW Grainger Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Trading Companies and Distributors,https://static.finnhub.io/logo/f153dcda-80eb-1...,https://www.grainger.com/,B,Medium,BB,...,B,Medium,255,385,240,880,19-04-2022,BB,Medium,277135
3,mhk,Mohawk Industries Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Consumer products,https://static.finnhub.io/logo/26868a62-80ec-1...,https://mohawkind.com/,A,High,B,...,BB,Medium,570,298,303,1171,18-04-2022,BBB,High,851968
4,lyv,Live Nation Entertainment Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,https://static.finnhub.io/logo/1cd144d2-80ec-1...,https://www.livenationentertainment.com/,BBB,High,BB,...,B,Medium,492,310,250,1052,18-04-2022,BBB,High,1335258


In [32]:
df.drop(columns = ["logo", "weburl", "environment_grade", "environment_level", "social_grade", "social_level", "governance_grade", "governance_level", "total_grade", "total_level", "cik", "last_processing_date"], inplace = True)

In [33]:
df.head(3)

,ticker,name,currency,exchange,industry,environment_score,social_score,governance_score,total_score
0,dis,Walt Disney Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,510,316,321,1147
1,gm,General Motors Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Automobiles,510,303,255,1068
2,gww,WW Grainger Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Trading Companies and Distributors,255,385,240,880


In [34]:
df.isna().any()

,0
ticker,False
name,False
currency,False
exchange,False
industry,True
environment_score,False
social_score,False
governance_score,False
total_score,False


In [35]:
row_na = df[df["industry"].isna()]

In [36]:
print(row_na["name"])

15                Armada Acquisition Corp I
27            Acri Capital Acquisition Corp
32         ACE Convergence Acquisition Corp
57                    Edoc Acquisition Corp
76                      AF Acquisition Corp
97                     AIB Acquisition Corp
101        Sports Ventures Acquisition Corp
123                Alignment Healthcare LLC
630       Health Assurance Acquisition Corp
646    Healthcare Services Acquisition Corp
669                Artisan Acquisition Corp
675                          Powered Brands
696                Concord Acquisition Corp
Name: name, dtype: object


In [37]:
industry_map = {
    'Armada Acquisition Corp I': 'Financial Services',
    'Acri Capital Acquisition Corp': 'Financial Services',
    'ACE Convergence Acquisition Corp': 'Technology',
    'Edoc Acquisition Corp': 'Healthcare',
    'AF Acquisition Corp': 'Financial Services',
    'AIB Acquisition Corp': 'Financial Services',
    'Sports Ventures Acquisition Corp': 'Media & Entertainment',
    'Alignment Healthcare LLC': 'Healthcare',
    'Health Assurance Acquisition Corp': 'Healthcare',
    'Healthcare Services Acquisition Corp': 'Healthcare',
    'Artisan Acquisition Corp': 'Financial Services',
    'Powered Brands': 'Consumer Goods',
    'Concord Acquisition Corp': 'Financial Services'
}
df['industry'] = df.apply(
    lambda row: industry_map[row['name']] if pd.isna(row['industry']) and row['name'] in industry_map else row['industry'],
    axis=1
)


In [38]:
df.isna().any()
# ESG Dataset cleaned

,0
ticker,False
name,False
currency,False
exchange,False
industry,False
environment_score,False
social_score,False
governance_score,False
total_score,False


In [39]:
df["Ticker"] = df["ticker"].str.replace('$', '', regex=False).str.upper()
tickers = df["Ticker"].unique()  # Avoid redundant API calls for repeated tickers

latest_prices = {}

for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="1d")
        if not hist.empty:
            latest_prices[ticker] = hist["Close"].iloc[-1]
        else:
            print(f"{ticker} has no data.")
            latest_prices[ticker] = None
    except Exception as e:
        print(f"{ticker} failed: {e}")
        latest_prices[ticker] = None

# Convert to DataFrame
latest_price_df = pd.DataFrame(list(latest_prices.items()), columns=["Ticker", "Latest_Price"])

# Merge on cleaned "Ticker" column
stock_merged = pd.merge(df, latest_price_df, on="Ticker", how="inner")

print(stock_merged.head())

DIS failed: name 'yf' is not defined
GM failed: name 'yf' is not defined
GWW failed: name 'yf' is not defined
MHK failed: name 'yf' is not defined
LYV failed: name 'yf' is not defined
LVS failed: name 'yf' is not defined
CLX failed: name 'yf' is not defined
AACG failed: name 'yf' is not defined
AAL failed: name 'yf' is not defined
AAME failed: name 'yf' is not defined
AAOI failed: name 'yf' is not defined
AAON failed: name 'yf' is not defined
AAPL failed: name 'yf' is not defined
AATC failed: name 'yf' is not defined
AAWW failed: name 'yf' is not defined
AACI failed: name 'yf' is not defined
AADI failed: name 'yf' is not defined
ABEO failed: name 'yf' is not defined
ABNB failed: name 'yf' is not defined
ABIO failed: name 'yf' is not defined
ABMD failed: name 'yf' is not defined
ABOS failed: name 'yf' is not defined
ABSI failed: name 'yf' is not defined
ABTX failed: name 'yf' is not defined
ABUS failed: name 'yf' is not defined
ABVC failed: name 'yf' is not defined
ACAD failed: name 'yf